In [84]:
import pandas as pd
import numpy as np

In [85]:
c19=pd.read_excel('datasets/C-19.xlsx',skiprows=6,header=None,engine='openpyxl')
c19.fillna(0,inplace=True)

-Illiterate
-literate
-Below Primary
-primary
-Middle
-Matric and Secondary+HS+Non tech+Tech as (Matric but below graduate)
-Grad and above

In [86]:
c08=pd.read_excel('datasets/C-08.xlsx',engine='openpyxl',skiprows=7,header=None)

## obtain useful data from C-08 and C-19 for age-groups

- formulas used:

In [87]:
# STATE_NAMES=[list(np.unique(c18.iloc[:,2].values))]
STATE_NAMES=[]
for state in c19.iloc[:,2].values:
    if not (state in STATE_NAMES) and state!=0:
        STATE_NAMES.append(state)

# although it is a bit of manual work but it is worth the efforts
LITRATE_GROUPS=['Illiterate', 'Literate but below primary', 'Primary but below middle', 'Middle but below matric/secondary', 'Matric/Secondary but below graduate', 'Graduate and above']
LITRATE_GROUP_COLS=[[[10,11]],[[19,20]],[[22,23]],[[25,26]],[[28,29],[31,32],[34,35],[37,38]],[[40,41]]]

In [88]:
for j in range(6):
    for litracy_list in LITRATE_GROUP_COLS[j]:
        print(litracy_list[1])

11
20
23
26
29
32
35
38
41


In [89]:
c08[(c08.iloc[:,1]==0) & (c08.iloc[:,4]=='Total') & (c08.iloc[:,5]=='All ages')].iloc[0,[10,11][1]]

258709529

In [90]:
useful_data=[]
for i,state in enumerate(STATE_NAMES):
    foo_df=c08[(c08.iloc[:,1]==i) & (c08.iloc[:,4]=='Total') & (c08.iloc[:,5]=='All ages')]
    for j,litracy_grp in enumerate(LITRATE_GROUPS):
        male_pop=0
        female_pop=0
        for litracy_list in LITRATE_GROUP_COLS[j]:
            male_pop+=foo_df.iloc[0,litracy_list[0]]
            female_pop+=foo_df.iloc[0,litracy_list[1]]
    
        # tri
        tri_male=c19[(c19.iloc[:,0]==str(i).zfill(2)) & (c19.iloc[:,4]==litracy_grp) & (c19.iloc[:,3]=='Total')].iloc[0,9]
        tri_female=c19[(c19.iloc[:,0]==str(i).zfill(2)) & (c19.iloc[:,4]==litracy_grp) & (c19.iloc[:,3]=='Total')].iloc[0,10]

        #bi
        bi_male=c19[(c19.iloc[:,0]==str(i).zfill(2)) & (c19.iloc[:,4]==litracy_grp) & (c19.iloc[:,3]=='Total')].iloc[0,6] - tri_male
        bi_female=c19[(c19.iloc[:,0]==str(i).zfill(2)) & (c19.iloc[:,4]==litracy_grp) & (c19.iloc[:,3]=='Total')].iloc[0,7] - tri_female

        #uni
        uni_male=male_pop-bi_male-tri_male
        uni_female=female_pop-bi_female-tri_female

        item={
            'state-code':i,
            'state-name':state,
            'litracy-group':litracy_grp,
            'litracy-group-male-pop':male_pop,
            'litracy-group-female-pop':female_pop,
            'tri-male-ratio':tri_male/male_pop,
            'tri-female-ratio':tri_female/female_pop,
            'bi-male-ratio':bi_male/male_pop,
            'bi-female-ratio':bi_female/female_pop,
            'uni-male-ratio':uni_male/male_pop,
            'uni-female-ratio':uni_female/female_pop
        }

        useful_data.append(item)

In [91]:
df=pd.DataFrame(useful_data)

## 3+ 

In [92]:
tri_list=[]
bi_list=[]
uni_list=[]
for i in range(36):
    male_values=df[df['state-code']==i].sort_values(by='tri-male-ratio',ascending=False).iloc[0,[2,5]].values
    female_values=df[df['state-code']==i].sort_values(by='tri-male-ratio',ascending=False).iloc[0,[2,6]].values
    tri_item={
        'state/ut':STATE_NAMES[i],
        'litracy-group-male':male_values[0],
        'ratio-of-3-male':male_values[1],
        'litracy-group-female':female_values[0],
        'ratio-of-3-female':female_values[1]
    }

    tri_list.append(tri_item)

    male_values=df[df['state-code']==i].sort_values(by='bi-male-ratio',ascending=False).iloc[0,[2,7]].values
    female_values=df[df['state-code']==i].sort_values(by='bi-male-ratio',ascending=False).iloc[0,[2,8]].values
    bi_item={
        'state/ut':STATE_NAMES[i],
        'litracy-group-male':male_values[0],
        'ratio-of-2-male':male_values[1],
        'litracy-group-female':female_values[0],
        'ratio-of-2-female':female_values[1]
    }

    bi_list.append(bi_item)

    male_values=df[df['state-code']==i].sort_values(by='uni-male-ratio',ascending=False).iloc[0,[2,9]].values
    female_values=df[df['state-code']==i].sort_values(by='uni-male-ratio',ascending=False).iloc[0,[2,10]].values
    uni_item={
        'state/ut':STATE_NAMES[i],
        'litracy-group-male':male_values[0],
        'ratio-of-1-male':male_values[1],
        'litracy-group-female':female_values[0],
        'ratio-of-1-female':female_values[1]
    }

    uni_list.append(uni_item)

In [93]:
tri_df=pd.DataFrame(tri_list)
bi_df=pd.DataFrame(bi_list)
uni_df=pd.DataFrame(uni_list)

In [94]:
tri_df.to_csv('outputs/litracy-gender-3.csv',index=False)
bi_df.to_csv('outputs/litracy-gender-2.csv',index=False)
uni_df.to_csv('outputs/litracy-gender-1.csv',index=False)

## observations

- in almost all cases both highest ratio female and male age-groups are same.
- interestingly in more than three language case for all states 'graduate and above' literacy group dominates, and it is also quite intutive; since they are more aducated and most of them speak more than 3+ langs 